In [2]:
import requests
from lxml import etree
#import pymysql
import re
import dataframe as df
import time
#db = pymysql.connect('localhost', 'root', '', 'KXO205' )

UQ_main_url = "https://my.uq.edu.au/programs-courses/program_list.html?acad_prog=5522"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36'
}

# 获取网页内容
def get_url(url, headers):
    session = requests.Session()
    resp = session.get(url, headers=headers)
    html = resp.text
    return html


def parse_uq_course_list(content):#分析所有课程的界面   https://my.uq.edu.au/programs-courses/program_list.html?acad_prog=5522
    hrefs = []  # 存放抓取的链接
    html = etree.HTML(content)
    table_list = html.xpath("//table")  # 主界面的Part ABC课程的table
    for table in table_list:
        trs = table.xpath("./tbody/tr")  # 每个table的所有tr
        for tr in trs:
            try:
                href = ""
                href = tr.xpath("./td[1]/a/@href")[0]  # 取出每个课程介绍的链接
            except Exception as e:
                pass
            if(href!=""):
                href = "https://my.uq.edu.au" + href
                hrefs.append(href)
    return hrefs


def parse_course_details(content):#获取每一个units的链接
    html = etree.HTML(content)
    try:
        #  获取该课程的学期
        title = html.xpath("//h1[@id='course-title']/text()")

        #  获取该课程的学期
        semester = html.xpath("//a[@id='course-offering-1-sem']/text()")
    except:
        semester = ""
        pass
    matchObj = re.search(r'.\d', semester[0], flags=0)
    if matchObj:
        semester =  matchObj.group(0)
    else:
        semester = None
    return title[0], semester

In [3]:
main_page = get_url(UQ_main_url,headers)
hrefs = parse_uq_course_list(main_page)

In [23]:
import pandas as pd

In [5]:
courses = {}
for href in hrefs:
    time.sleep(1)
    content = get_url(href, headers)
    title, semester = parse_course_details(content)
    courses.update({title: semester})

In [14]:
courses

{'Compilers and Interpreters (COMP7402)': ' 1',
 'Advanced Algorithms & Data Structures (COMP7500)': ' 2',
 'Machine Learning (COMP7703)': ' 1',
 'Communication Systems (COMS4105)': ' 2',
 'Computer Networks II (COMS7200)': ' 2',
 'Advanced topics in Security (COMS7507)': ' 1',
 'Distributed Computing (CSSE4004)': ' 1',
 'Advanced Embedded Systems (CSSE4011)': ' 1',
 'Principles of Program Analysis (CSSE4630)': ' 2',
 'Concurrency: Theory and Practice (CSSE7610)': ' 2',
 'Advanced Human-Computer Interaction (DECO6500)': ' 2',
 'Data Mining (INFS7203)': ' 2',
 'Advanced Techniques for High Dimensional Data (INFS7205)': ' 1',
 'Information Retrieval and Web Search (INFS7410)': ' 2',
 'Social Media Analytics (INFS7450)': ' 1',
 'Special Topics in Computer Science 7A (COMP7000)': ' 1',
 'Special Topics in Computer Science 7B (COMP7001)': ' 1',
 'Advanced Topics in Computer Systems A (CSSE7080)': ' 1',
 'Advanced Topics in Computer Systems B (CSSE7081)': ' 1',
 'Advanced Topics in Software 

In [19]:
title = []
semester = []
for key, value in courses.items():
    title.append(key)
    semester.append(value)
new_courses = {"title":title, "semester": semester}


{'title': ['Compilers and Interpreters (COMP7402)',
  'Advanced Algorithms & Data Structures (COMP7500)',
  'Machine Learning (COMP7703)',
  'Communication Systems (COMS4105)',
  'Computer Networks II (COMS7200)',
  'Advanced topics in Security (COMS7507)',
  'Distributed Computing (CSSE4004)',
  'Advanced Embedded Systems (CSSE4011)',
  'Principles of Program Analysis (CSSE4630)',
  'Concurrency: Theory and Practice (CSSE7610)',
  'Advanced Human-Computer Interaction (DECO6500)',
  'Data Mining (INFS7203)',
  'Advanced Techniques for High Dimensional Data (INFS7205)',
  'Information Retrieval and Web Search (INFS7410)',
  'Social Media Analytics (INFS7450)',
  'Special Topics in Computer Science 7A (COMP7000)',
  'Special Topics in Computer Science 7B (COMP7001)',
  'Advanced Topics in Computer Systems A (CSSE7080)',
  'Advanced Topics in Computer Systems B (CSSE7081)',
  'Advanced Topics in Software Engineering A (CSSE7090)',
  'Advanced Topics in Software Engineering B (CSSE7091)',


In [24]:
df = pd.DataFrame(new_courses)

                                                title semester
0               Compilers and Interpreters (COMP7402)        1
1    Advanced Algorithms & Data Structures (COMP7500)        2
2                         Machine Learning (COMP7703)        1
3                    Communication Systems (COMS4105)        2
4                     Computer Networks II (COMS7200)        2
5              Advanced topics in Security (COMS7507)        1
6                    Distributed Computing (CSSE4004)        1
7                Advanced Embedded Systems (CSSE4011)        1
8           Principles of Program Analysis (CSSE4630)        2
9         Concurrency: Theory and Practice (CSSE7610)        2
10     Advanced Human-Computer Interaction (DECO6500)        2
11                             Data Mining (INFS7203)        2
12  Advanced Techniques for High Dimensional Data ...        1
13    Information Retrieval and Web Search (INFS7410)        2
14                  Social Media Analytics (INFS7450)  

In [27]:
df[df["semester"]==" 2"]

,title,semester
1,Advanced Algorithms & Data Structures (COMP7500),2
3,Communication Systems (COMS4105),2
4,Computer Networks II (COMS7200),2
8,Principles of Program Analysis (CSSE4630),2
9,Concurrency: Theory and Practice (CSSE7610),2
10,Advanced Human-Computer Interaction (DECO6500),2
11,Data Mining (INFS7203),2
13,Information Retrieval and Web Search (INFS7410),2
22,Operating Systems Architecture (COMP7308),2
23,Algorithms & Data Structures (COMP7505),2


In [28]:
df[df["semester"]==" 1"]

,title,semester
0,Compilers and Interpreters (COMP7402),1
2,Machine Learning (COMP7703),1
5,Advanced topics in Security (COMS7507),1
6,Distributed Computing (CSSE4004),1
7,Advanced Embedded Systems (CSSE4011),1
12,Advanced Techniques for High Dimensional Data ...,1
14,Social Media Analytics (INFS7450),1
15,Special Topics in Computer Science 7A (COMP7000),1
16,Special Topics in Computer Science 7B (COMP7001),1
17,Advanced Topics in Computer Systems A (CSSE7080),1


In [36]:
for i in range(0, 21):
    df.loc[i,'Part'] = 'A'
for i in range(21, 35):
    df.loc[i,'Part'] = 'B'
for i in range(35, 45):
    df.loc[i,'Part'] = 'C'

In [49]:
df[df["Part"] == "A"].sort_values("semester")

,title,semester,Part
0,Compilers and Interpreters (COMP7402),1,A
18,Advanced Topics in Computer Systems B (CSSE7081),1,A
17,Advanced Topics in Computer Systems A (CSSE7080),1,A
16,Special Topics in Computer Science 7B (COMP7001),1,A
15,Special Topics in Computer Science 7A (COMP7000),1,A
14,Social Media Analytics (INFS7450),1,A
12,Advanced Techniques for High Dimensional Data ...,1,A
19,Advanced Topics in Software Engineering A (CSS...,1,A
20,Advanced Topics in Software Engineering B (CSS...,1,A
7,Advanced Embedded Systems (CSSE4011),1,A
